In [15]:
import numpy as np
import pandas as pd

# Métodos numéricos

## Método de bisección

In [14]:
def bisection(f, a, b, epsilon, max_iter=100):
    fa = f(a)
    fb = f(b)
    if fa * fb >= 0:
        raise ValueError("f(a) y f(b) deben tener signos opuestos")
    iters = []
    n = 1
    while ((b - a) > epsilon and n <= max_iter):
        x = (a + b) / 2
        fx = f(x)
        iters.append({
            "iter": n,
            "lim_inf": a,
            "lim_sup": b,
            "aprox": x,
            "error": (b-a) / 2,
            "residuo": abs(fx)
        })
        if fa * fx < 0:
            b = x
        else:
            a = x
            fa = fx
        n += 1

    return pd.DataFrame(iters)

In [13]:
bisection(lambda x: x**3 - 19, 0, 3, 1e-2)

,iter,lim_inf,lim_sup,aprox,error,residuo
0,1,0.000000,3.000000,1.500000,1.500000,15.625000
1,2,1.500000,3.000000,2.250000,0.750000,7.609375
2,3,2.250000,3.000000,2.625000,0.375000,0.912109
3,4,2.625000,3.000000,2.812500,0.187500,3.247314
4,5,2.625000,2.812500,2.718750,0.093750,1.095917
5,6,2.625000,2.718750,2.671875,0.046875,0.074291
6,7,2.625000,2.671875,2.648438,0.023438,0.423274
7,8,2.648438,2.671875,2.660156,0.011719,0.175587
8,9,2.660156,2.671875,2.666016,0.005859,0.050923


## Método de Newton

**Teorema (método de Newton)**

Sea $f:(a,b)\to\mathbb{R}$ una función de clase $C^2$ en $(a,b)$. Supongamos que en $(a,b)$:

1. $f'(x)$ tiene signo constante y no se anula en $(a,b)$ (crecimiento o decrecimiento estricto en el intervalo).
2. $f''(x)$ tiene signo constante y no se anula en $(a,b)$ (convexidad o concavidad global en el intervalo).
3. Existe una raíz $\alpha\in (a,b)$.
4. Se toma $x_0\in(a,b)$ con la condición de arranque $f(x_0)\,f''(x_0)>0$

Definimos la sucesión de Newton
$$x_{n+1}=x_n-\frac{f(x_n)}{f'(x_n)}$$

Entonces se cumplen:

1. **Unicidad de la raíz**: la raíz $\alpha$ en $(a,b)$ es única.
2. **Convergencia**: la sucesión $\{x_n\}$ converge a $\alpha$.
3. **Estimaciones del error**: si además existen constantes $0<m\le M$ tales que
   $$0<m\le |f'(x)|\le M \quad \text{para todo } x \text{ en un intervalo que contiene a } \alpha \text{ y a } x_n$$
   entonces para todo $n\ge 1$ se verifican
   $$|x_n-\alpha|\le \frac{|f(x_n)|}{m},\qquad|x_n-\alpha|\le \frac{M-m}{m}\,|x_n-x_{n-1}|$$

### Demostración

Como $f'$ tiene signo constante y no se anula, $f$ es estrictamente monótona en $(a,b)$.
Una función estrictamente monótona puede cortar el eje $x$ a lo sumo una vez.
Por tanto, si existe una raíz $\alpha\in[a,b]$, es única.

Sin pérdida de generalidad, demostramos el caso típico
$$f'(x)>0 \quad\text{y}\quad f''(x)>0 \ \ \text{en } (a,b)$$
(es decir, $f$ creciente y convexa). Los otros casos (signos cambiados) son análogos invirtiendo desigualdades.

Para $f$ convexa, para todo $x,y\in(a,b)$ se cumple
$$f(y)\ge f(x)+f'(x)(y-x)$$
Tomando $y=\alpha$ (con $f(\alpha)=0$) queda
$$0\ge f(x)+f'(x)(\alpha-x)\quad\Longrightarrow\quad f'(x)(x-\alpha)\ge f(x)$$
Dividiendo por $f'(x)>0$,
$$x-\frac{f(x)}{f'(x)}\ge \alpha$$
Pero $x-\frac{f(x)}{f'(x)}=N(x)$ es el operador de Newton, luego
$$x_{n+1}\ge \alpha\quad\text{siempre que }x_n\in(a,b)$$

La condición $f(x_0)f''(x_0)>0$ y $f''>0$ implica $f(x_0)>0$. Como $f$ es creciente y $f(\alpha)=0$, esto fuerza $x_0>\alpha$.

Si $x_n>\alpha$, entonces $f(x_n)>0$ y $f'(x_n)>0$, por tanto
$$x_{n+1}=x_n-\frac{f(x_n)}{f'(x_n)}<x_n$$
Luego
$$\alpha \le x_{n+1} < x_n$$
Así, $\{x_n\}$ es decreciente y está acotada inferiormente por $\alpha$, luego converge:
existe $\ell=\lim x_n$ con $\ell\ge \alpha$.

Pasando a límite en la iteración (usando continuidad de $f$ y $f'$ y que $f'(x)\neq 0$):
$$\ell=\ell-\frac{f(\ell)}{f'(\ell)} \quad\Longrightarrow\quad f(\ell)=0$$
Por unicidad de la raíz, $\ell=\alpha$. Se concluye que $x_n\to \alpha$.

A partir de aquí suponemos que en el intervalo donde viven $\alpha$ y los iterados se cumple
$$0<m\le |f'(x)|\le M$$

Por el Teorema del Valor Medio entre $x_n$ y $\alpha$, existe $\xi_n$ entre ambos tal que
$$f(x_n)-f(\alpha)=f'(\xi_n)(x_n-\alpha)$$
Como $f(\alpha)=0$,
$$|f(x_n)|=|f'(\xi_n)|\,|x_n-\alpha|\ge m\,|x_n-\alpha|$$
Despejando,
$$|x_n-\alpha|\le \frac{|f(x_n)|}{m}$$
quedando demostrada la primera estimación.

Ahora aplicamos el TVM a $f(x_{n-1})-f(\alpha)$: existe $\eta_{n-1}$ entre $x_{n-1}$ y $\alpha$ tal que
$$f(x_{n-1})=f'(\eta_{n-1})(x_{n-1}-\alpha)$$
Definimos
$$r:=\frac{f'(\eta_{n-1})}{f'(x_{n-1})}$$
donde el cociente tiene sentido porque $f'(x_{n-1})\neq 0$.
A partir de la fórmula de Newton:
$$x_n-x_{n-1}=-\frac{f(x_{n-1})}{f'(x_{n-1})}=-\frac{f'(\eta_{n-1})}{f'(x_{n-1})}(x_{n-1}-\alpha)=-r\,(x_{n-1}-\alpha)$$
De aquí, tomando valores absolutos,
$$|x_n-x_{n-1}|=r\,|x_{n-1}-\alpha|$$
Además
$$x_n-\alpha=(x_{n-1}-\alpha)+(x_n-x_{n-1})=(1-r)(x_{n-1}-\alpha)$$
luego
$$|x_n-\alpha|=(1-r)\,|x_{n-1}-\alpha|$$
Dividiendo ambas expresiones:
$$\frac{|x_n-\alpha|}{|x_n-x_{n-1}|}=\frac{1-r}{r}$$

Ahora acotamos $r$. Como $|f'(\eta_{n-1})|\ge m$ y $|f'(x_{n-1})|\le M$,
$$r=\frac{|f'(\eta_{n-1})|}{|f'(x_{n-1})|}\ge \frac{m}{M}$$
Además, dado que estamos suponiendo $f''>0$, en $(a,b)$, $f'$ es creciente y como $\alpha\leq\eta_{n-1}\leq x_{n-1}$ se tiene que $r\le 1$.
Por tanto $r\in\left[\frac{m}{M},1\right]$, y como $\phi(r)=\frac{1-r}{r}=\frac1r-1$ es decreciente,
$$\frac{1-r}{r}\le \frac{1-\frac{m}{M}}{\frac{m}{M}}=\frac{M-m}{m}$$
Concluimos,
$$|x_n-\alpha|\le \frac{M-m}{m}\,|x_n-x_{n-1}|$$
Queda demostrada la segunda estimación.

## Diferenciación numérica

Sea $f:(a,b)\longrightarrow \mathbb{R}$ una función que suponemos **suficientemente regular**, lo que significa que las derivadas de $f$ involucradas en el contexto en el que estamos existen y son continuas. Considérese también un punto $x_0\in (a,b)$.

En la práctica **queremos** aproximar el valor de su derivada $f'(x_0)$ y, en muchas ocasiones, no disponemos de ningún método analítico que permita aplicar directamente las fórmulas de cálculo conocidas. Por ello, se necesitan **fórmulas numéricas de cómputo**. En primer lugar, veamos que, de hecho, a partir de la propia definición de derivada
$$
f'(x_0)=\lim_{h\to 0}\frac{f(x_0+h)-f(x_0)}{h},
$$
podemos obtener de manera natural una fórmula de diferenciación numérica. Definimos el siguiente cociente
$$
f'_{h,+}(x_0)=\frac{f(x_0+h)-f(x_0)}{h},\quad h>0.
$$
Por definición, se tiene que
$$
f'_{h,+}(x_0)\to f'(x_0)\quad \text{cuando } h\to 0^+.
$$
Luego $f'_{h,+}(x_0)$ proporciona una aproximación de la derivada cuando $h\approx 0$ es suficientemente pequeño. A la expresión anterior se la denomina **fórmula de diferencias finitas progresiva** y, en general, al parámetro positivo $h$ lo denominaremos **tamaño de paso** o **parámetro de discretización**.

Es posible obtener una estimación del error cometido al aproximar $f'(x_0)$ por $f'_{h,+}(x_0)$. Para ello, aplicando el teorema de Taylor se obtiene
$$
f'_{h,+}(x_0)
=\frac{f(x_0+h)-f(x_0)}{h}
=\frac{f'(x_0)h+\frac{1}{2}f''(c)h^2}{h}
=f'(x_0)+\frac{1}{2}f''(c)h,
\quad c\in(x_0,x_0+h).
$$
Por tanto,
$$
|f'*{h,+}(x_0)-f'(x_0)|
=\frac{1}{2}|f''(c)|h
\le kh,
\quad \text{donde } k=\sup_{x\in(a,b)}\frac{1}{2}|f''(x)|.
$$
Lo anterior indica que el error cometido al aproximar $f'(x_0)$ por $f'*{h,+}(x_0)$ tiende a $0$ cuando $h\to 0$ al menos tan rápido como la función $g(h)=h$. Cuando esto sucede, diremos que el error es de **orden $h$**, y lo denotaremos por
$$
E(h)=|f'_{h,+}(x_0)-f'(x_0)|=O(h).
$$

Existen numerosos tipos de fórmulas de diferencias finitas para calcular la derivada primera, con distintos órdenes de convergencia:

| Método                       | Fórmula                                       | Error    |
| ---------------------------- | --------------------------------------------- | -------- |
| Diferencia finita progresiva | $f'_{h,+}(x_0)=\dfrac{f(x_0+h)-f(x_0)}{h}$    | $O(h)$   |
| Diferencia finita regresiva  | $f'_{h,-}(x_0)=\dfrac{f(x_0)-f(x_0-h)}{h}$    | $O(h)$   |
| Diferencia finita centrada   | $f'_{h,c}(x_0)=\dfrac{f(x_0+h)-f(x_0-h)}{2h}$ | $O(h^2)$ |

Finalmente, comentamos que se puede seguir un desarrollo análogo para obtener fórmulas de aproximación de derivadas de cualquier orden. Algunas de las fórmulas más usuales de diferencias finitas para la segunda derivada son las siguientes:

| Método                       | Fórmula                                                  | Error    |
| ---------------------------- | -------------------------------------------------------- | -------- |
| Diferencia finita progresiva | $f''_{h,+}(x_0)=\dfrac{f(x_0+2h)-2f(x_0+h)+f(x_0)}{h^2}$ | $O(h)$   |
| Diferencia finita regresiva  | $f''_{h,-}(x_0)=\dfrac{f(x_0)-2f(x_0-h)+f(x_0-2h)}{h^2}$ | $O(h)$   |
| Diferencia finita centrada   | $f''_{h,c}(x_0)=\dfrac{f(x_0+h)-2f(x_0)+f(x_0-h)}{h^2}$  | $O(h^2)$ |



In [16]:
def diff_prog(f, x_0, h=1e-2):
    if (h <= 0):
        raise ValueError("El tamaño de paso h no puede ser negativo")
    return (f(x_0+h)-f(x_0))/h

In [23]:
def diff_cent(f, x_0, h=1e-2):
    if (h <= 0):
        raise ValueError("El tamaño de paso h no puede ser negativo")
    return (f(x_0+h)-f(x_0-h))/(2*h)

In [34]:
f = lambda x: np.exp(x)
pasos = [1e-1, 1e-2, 1e-3, 1e-4]
x_0 = 1
df_exacta = np.exp(x_0)
data = []
for h in pasos:
    data.append({
        "paso": h,
        "progresiva": diff_prog(f, x_0, h),
        "centrada": diff_cent(f, x_0, h)
    })
df_comp = pd.DataFrame(data)
df_comp['error_progresiva'] = np.abs(df_comp['progresiva']-df_exacta)
df_comp['error_progresiva/h'] = df_comp['error_progresiva'] / df_comp['paso']
df_comp['error_centrada'] = np.abs(df_comp['centrada']-df_exacta)
df_comp['error_centrada/h^2'] = df_comp['error_centrada'] / df_comp['paso'] ** 2

df_comp

,paso,progresiva,centrada,error_progresiva,error_progresiva/h,error_centrada,error_centrada/h^2
0,0.1000,2.858842,2.722815,0.140560,1.405601,4.532735e-03,0.453274
1,0.0100,2.731919,2.718327,0.013637,1.363683,4.530492e-05,0.453049
2,0.0010,2.719641,2.718282,0.001360,1.359594,4.530467e-07,0.453047
3,0.0001,2.718418,2.718282,0.000136,1.359186,4.530566e-09,0.453057


,paso,progresiva,centrada,error_progresiva
0,0.1000,2.858842,2.722815,0.140560
1,0.0100,2.731919,2.718327,0.013637
2,0.0010,2.719641,2.718282,0.001360
3,0.0001,2.718418,2.718282,0.000136
